<a href="https://colab.research.google.com/github/lynnlo/CB1/blob/master/CB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import LSTM, BatchNormalization, Embedding, Bidirectional, Dropout, InputLayer, Flatten, Dense, Reshape
from matplotlib import pyplot

import numpy

In [2]:
# Data
from urllib.request import urlopen

try:
    file = open('local_data.txt').read()
except:
    file = urlopen("https://raw.githubusercontent.com/lynnlo/CB1/master/dialogs.txt").read().decode('utf-8')
    local = open('local_data.txt', 'w')
    local.write(file)
    local.close()

In [3]:
# Split Data
file = file.replace("\n", "\t")
data_raw = file.split("\t")

# Data
data = []
input, output = [], []
dictionary = { '<ov>': 0, ' ': 1 }
word_list = ['<ov>', ' ']

# Switch for spliting array in groups of 2.
t = False

# Sizing Params
max_size = 0
total_data_size = 5000

for x in data_raw:
    if t:
        data[-1].append(x)
        t = not t
    else:
        data.append([x])
        t = not t

In [4]:
for line in data[0:total_data_size]:
    # Input
    input_sentence = []
    for word in line[0].split(' '):
        if dictionary.get(word, 0) == 0:
            dictionary[word] = len(dictionary)
            word_list.append(word)
        input_sentence.append(word)
    # Update Max Size
    if len(input_sentence) > max_size:
        max_size = len(input_sentence)
    input.append(input_sentence) # Add sentence

    # Output
    output_sentence = []
    for word in line[1].split(' '):
        if dictionary.get(word, 0) == 0:
            dictionary[word] = len(dictionary)
            word_list.append(word)
        output_sentence.append(word)
    # Update Max Size
    if len(input_sentence) > max_size:
        max_size = len(input_sentence)
    output.append(output_sentence) # Add sentence


In [5]:
for sentence in input:
    while len(sentence) < max_size:
        sentence.append(' ')

for sentence in output:
    while len(sentence) < max_size:
        sentence.append(' ')

In [6]:
input_vectorized = []
output_vectorized = []

for sentence in input:
    input_sentence_vectorized = []
    for word in sentence:
        input_sentence_vectorized.append(dictionary.get(word))
    input_vectorized.append(input_sentence_vectorized)

for sentence in output:
    output_sentence_vectorized = []
    for word in sentence:
        output_sentence_vectorized.append(dictionary.get(word))
    output_vectorized.append(output_sentence_vectorized)

In [7]:
input_encoded = []
output_encoded = []

for x in input_vectorized:
    input_encoded.append([])
    for i,v in enumerate(x):
        ohe = numpy.zeros(len(dictionary))
        ohe[v] = 1
        input_encoded[-1].append(ohe)

for x in output_vectorized:
    output_encoded.append([])
    for i,v in enumerate(x):
        ohe = numpy.zeros(len(dictionary))
        ohe[v] = 1
        output_encoded[-1].append(ohe)

input_encoded = numpy.array(input_encoded)
output_encoded = numpy.array(output_encoded)

input_encoded = input_encoded.reshape(-1, max_size, len(dictionary))
output_encoded = output_encoded.reshape(-1, max_size, len(dictionary))

In [8]:
input_encoded.shape

(3725, 19, 4042)

In [9]:
model = keras.Sequential()

#model.add(Embedding(len(word_list) + 1, len(word_list)))

model.add(LSTM(len(word_list), return_sequences=True, input_shape=(max_size, len(word_list))))

model.compile(optimizer='adam', loss='mse')

In [ ]:
total_history = []
model.build(input_encoded.shape)

model.summary()

In [ ]:
model.fit(x=input_encoded, y=output_encoded, epochs=5, batch_size=64, validation_split=0.05)

In [ ]:
total_history = numpy.concatenate((total_history, model.history.history["loss"]))

In [ ]:
pyplot.plot(total_history)

pyplot.show()

In [ ]:
def predict(n):
    prediction = model.predict(input_encoded[n].reshape(1, max_size, len(word_list))).astype("float64")
    prediction_cleaned = []
    prediction_readable = []
    s = 0
    for i in prediction[0][0]:
        s += i
    print(s)

    for x in prediction:
        prediction_line = []
        for y in x:
            prediction_line.append(numpy.argmax(y))
        prediction_cleaned.append(prediction_line)

    for x in prediction_cleaned[0]:
        prediction_readable.append(word_list[x])

    print(prediction_cleaned[0])

    print(" ".join(input[n]))
    print(" ".join(output[n]))
    print(" ".join(prediction_readable))
    print("\n\n")

for i in range(10):
    predict(i)